In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pprint

import engage_geophires_client
import engage_parametrization_visualization

import importlib
importlib.reload(engage_geophires_client)
importlib.reload(engage_parametrization_visualization)

from engage_geophires_client import *
from engage_parametrization_visualization import *

## Notes
-  Flag error if the reservoir heat content is depleted before the project lifetime
-  EGS gradient sections need to be reviewed if there is more than one gradient
-  User should use only one value for all or define all four. If user has 50 gradient 1 and 75 gradient 2, 75 should be assigned to the 3 and 4
- Flash steam power plants are relatively a common method used to convert the geothermal energy into electricity when production wells produce a mixture of steam and liquid in a geothermal system. In comparison to the single flash system, the double flash steam power plant generates more power from geothermal fluid at the same conditions. However, electricity generation costs are higher for double flash plants. A thermoeconomic optimization model is presented in this paper as the generation costs of single and double flash steam power plants. By considering the pressures in the separator and flash vessel (only in a double flash plant) as the independent variables in the objective function and using numerical search methods (Golden Ratio and Nelder-Mead ), the minimum power generation cost was calculated at different geofluid conditions for either single or double flash steam power plants.
- Might need to implement a time-varying efficiency of the chiller based on yearly ambient temperature
- Flag Flash is only suitable after this depth (specify depth)
- Add injection temperature as an input to the engage user 
- Lifetime should be an input not an output
- Flash systems tell user to keep prod temperature target above 200


In [ ]:
def generate_parameters(base_params):
    parameter_list = []
    for depth in np.arange(1, 5, 0.1):
        # for plant_type in range(1,2):
        for flow_rate in range(50,150,50):
            for wells_prod in range(1, 5):
                for wells_inj in range(1, 5):
                    if wells_inj < wells_prod:
                        continue

                    # Create a copy of the base parameters and update specific values
                    params = base_params.copy()
                    params.update(
                        {
                            # 'Power Plant Type':plant_type,
                            'Production Flow Rate per Well':flow_rate,
                            'Reservoir Depth': depth,
                            'Number of Production Wells': wells_prod,
                            'Number of Injection Wells': wells_inj,
                        }
                    )
                    parameter_list.append(params)
    return parameter_list


# Define base parameters for each technology#
base_params = {
    'HYDRO_chp': {
        'Reservoir Model': 4,
        'Drawdown Parameter': 0.003,
        'Number of Segments': 1,
        'Gradient 1': 55,
        'Maximum Temperature': 400,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Ramey Production Wellbore Model': 0,
        'Production Wellbore Temperature Drop': 5,
        'Injection Wellbore Temperature Gain': 3,
        'Reservoir Volume Option': 1,
        'Injectivity Index': 5,
        'Injection Temperature': 70,
        'Maximum Drawdown': 1,
        'Reservoir Heat Capacity': 1000,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3,
        'Water Loss Fraction': 0.02,
        'End-Use Option': 31,
        'Power Plant Type': 1,
        'Circulation Pump Efficiency': 0.80,
        'Plant Lifetime': 30,
        'Well Drilling Cost Correlation': 10,				
        'Print Output to Console': 0
    },
    'EGS_chp': {
        'Reservoir Model':3, 
        'Drawdown Parameter': 0.00002,
        'Number of Segments': 4,
        'Gradient 1':55,
        'Thickness 1':1,
        'Gradient 2':55,
        'Thickness 2':1,
        'Gradient 3':55, 
        'Thickness 3':1,
        'Gradient 4':55,
        'Maximum Temperature': 400,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Injection Temperature': 50,
        'Reservoir Heat Capacity': 1000,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3,
        'End-Use Option': 31,
        'Circulation Pump Efficiency': 0.80,
        'Plant Lifetime': 30,
        'Fracture Shape': 3,                   
        'Fracture Height': 600,
        'Number of Fractures':20, 				
        'Reservoir Volume Option': 1, 
        'Power Plant Type': 1,
        'Well Drilling Cost Correlation': 10,				
        'Print Output to Console': 0
    },
    'HYDRO_binary_orc': {
        'Reservoir Model': 4,
        'Drawdown Parameter': 0.003,
        'Number of Segments': 1,
        'Gradient 1': 55,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Ramey Production Wellbore Model': 0,
        'Production Wellbore Temperature Drop': 0,
        'Injection Wellbore Temperature Gain': 0,
        'Maximum Temperature': 400,
        'Reservoir Volume Option': 4,
        'Reservoir Volume': 1e9,
        'Water Loss Fraction': 0.0,
        'Injectivity Index': 10,
        'Productivity Index': 10,
        'Injection Temperature': 70,
        'Maximum Drawdown': 1,
        'Reservoir Heat Capacity': 1050,
        'Reservoir Density': 2700,
        'Reservoir Thermal Conductivity': 3,
        'End-Use Option': 1,
        'Power Plant Type': 1,
        'Circulation Pump Efficiency': 0.8,
        'Utilization Factor': 0.9,
        'Surface Temperature': 15,
        'Ambient Temperature': 15,
        'Plant Lifetime': 30,
        'Well Drilling Cost Correlation': 10,
        'Print Output to Console': 0,
    },
    'EGS_binary_orc': {
        'Reservoir Model':3, 
        'Drawdown Parameter': 0.00002,
        'Number of Segments': 4,
        'Gradient 1':55,
        'Thickness 1':1,
        'Gradient 2':55,
        'Thickness 2':1,
        'Gradient 3':55, 
        'Thickness 3':1,
        'Gradient 4':55,
        'Maximum Temperature': 400,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Injection Temperature': 50,
        'Reservoir Heat Capacity': 1000,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3,
        'End-Use Option': 1,
        'Power Plant Type': 1,
        'Circulation Pump Efficiency': 0.80,
        'Plant Lifetime': 30,
        'Fracture Shape': 3,                   
        'Fracture Height': 600,
        'Number of Fractures':20, 				
        'Reservoir Volume Option': 1, 
        'Well Drilling Cost Correlation': 10,				
        'Print Output to Console': 0
    },
    'EGS_direct_use': {
        'Reservoir Model': 1,
        'Number of Segments': 1,
        'Gradient 1': 30,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Ramey Production Wellbore Model': 1,
        'Injection Wellbore Temperature Gain': 0,
        'Fracture Shape': 3,
        'Fracture Height': 700,
        'Reservoir Volume Option': 1,
        'Fracture Separation': 100,
        'Reservoir Impedance': 0.05,
        'Injection Temperature': 30,
        'Maximum Drawdown': 0.3,
        'Reservoir Heat Capacity': 975,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3.2,
        'End-Use Option': 2,
        'Circulation Pump Efficiency': 0.8,
        'End-Use Efficiency Factor': 0.9,
        'Surface Temperature': 20,
        'Ambient Temperature': 20,
        'Plant Lifetime': 30,
        'Power Plant Type': 1,
        'Well Drilling Cost Correlation': 10,
        'Print Output to Console': 0
    },
    'HYDRO_direct_use': {
        'Reservoir Model': 4,
        'Drawdown Parameter': 0.003,
        'Number of Segments': 1,
        'Gradient 1': 30,
        'Maximum Temperature': 400,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Ramey Production Wellbore Model': 0,
        'Production Wellbore Temperature Drop': 5,
        'Injection Wellbore Temperature Gain': 3,
        'Reservoir Volume Option': 1,
        'Injectivity Index': 5,
        'Injection Temperature': 70,
        'Maximum Drawdown': 1,
        'Reservoir Heat Capacity': 1000,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3,
        'Water Loss Fraction': 0.02,
        'End-Use Option': 2,
        'Circulation Pump Efficiency': 0.80,
        'Plant Lifetime': 30,
        'Power Plant Type': 1,
        'Well Drilling Cost Correlation': 10,				
        'Print Output to Console': 0
    },
    'HYDRO_double_flash': {
        'Reservoir Model': 4,
        'Drawdown Parameter': 0.003,
        'Number of Segments': 1,
        'Gradient 1': 75,
        'Maximum Temperature': 400,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Ramey Production Wellbore Model': 0,
        'Production Wellbore Temperature Drop': 5,
        'Injection Wellbore Temperature Gain': 3,
        'Reservoir Volume Option': 1,
        'Injectivity Index': 5,
        'Injection Temperature': 70,
        'Maximum Drawdown': 1,
        'Reservoir Heat Capacity': 1000,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3,
        'Water Loss Fraction': 0.02,
        'End-Use Option': 1,
        'Circulation Pump Efficiency': 0.80,
        'Plant Lifetime': 30,
        'Power Plant Type': 4,
        'Well Drilling Cost Correlation': 10,				
        'Print Output to Console': 0
},
    'EGS_double_flash': {
        'Reservoir Model':3, 
        'Drawdown Parameter': 0.00002,
        'Number of Segments': 4,
        'Gradient 1':75,
        'Thickness 1':1,
        'Gradient 2':75,
        'Thickness 2':1,
        'Gradient 3':75, 
        'Thickness 3':1,
        'Gradient 4':75,
        'Maximum Temperature': 400,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Injection Temperature': 50,
        'Reservoir Heat Capacity': 1000,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3,
        'End-Use Option': 1,
        'Circulation Pump Efficiency': 0.80,
        'Plant Lifetime': 30,
        'Fracture Shape': 3,                   
        'Fracture Height': 600,
        'Number of Fractures':20, 				
        'Reservoir Volume Option': 1, 
        'Power Plant Type': 4,
        'Well Drilling Cost Correlation': 10,				
        'Print Output to Console': 0
    },
    'HYDRO_cchp': {
        'Reservoir Model': 4,
        'Drawdown Parameter': 0.003,
        'Number of Segments': 1,
        'Gradient 1': 55,
        'Maximum Temperature': 400,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Ramey Production Wellbore Model': 0,
        'Production Wellbore Temperature Drop': 5,
        'Injection Wellbore Temperature Gain': 3,
        'Reservoir Volume Option': 1,
        'Injectivity Index': 5,
        'Injection Temperature': 70,
        'Maximum Drawdown': 1,
        'Reservoir Heat Capacity': 1000,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3,
        'Water Loss Fraction': 0.02,
        'End-Use Option': 31,
        'Power Plant Type': 1,
        'Circulation Pump Efficiency': 0.80,
        'Plant Lifetime': 30,
        'Well Drilling Cost Correlation': 10,				
        'Print Output to Console': 0
    },
    'EGS_cchp': {
        'Reservoir Model':3, 
        'Drawdown Parameter': 0.00002,
        'Number of Segments': 4,
        'Gradient 1':55,
        'Thickness 1':1,
        'Gradient 2':55,
        'Thickness 2':1,
        'Gradient 3':55, 
        'Thickness 3':1,
        'Gradient 4':55,
        'Maximum Temperature': 400,
        'Production Well Diameter': 8.5,
        'Injection Well Diameter': 8.5,
        'Injection Temperature': 50,
        'Reservoir Heat Capacity': 1000,
        'Reservoir Density': 3000,
        'Reservoir Thermal Conductivity': 3,
        'End-Use Option': 31,
        'Circulation Pump Efficiency': 0.80,
        'Plant Lifetime': 30,
        'Fracture Shape': 3,                   
        'Fracture Height': 600,
        'Number of Fractures':20, 				
        'Reservoir Volume Option': 1, 
        'Power Plant Type': 1,
        'Well Drilling Cost Correlation': 10,				
        'Print Output to Console': 0
    },
}

# Update the technology_to_function dictionary to include the new naming
technology_to_function = {
    'EGS_chp': generate_parameters,
    'HYDRO_chp': generate_parameters,
    'HYDRO_binary_orc': generate_parameters,
    'EGS_binary_orc': generate_parameters,
    'HYDRO_double_flash': generate_parameters,
    'EGS_double_flash': generate_parameters,
    'HYDRO_direct_use': generate_parameters,
    'EGS_direct_use': generate_parameters,
    'EGS_cchp': generate_parameters,
    'HYDRO_cchp': generate_parameters,
}

# Example usage
environment     = 'HYDRO'  # HYDRO OR EGS
technology      = 'chp'
plant           = f'{environment}_{technology}'

# Medium 
target_prod_temp_min   = 150
target_prod_temp_max   = 200


# Generate parameters based on the selected technology and environment
if plant in technology_to_function and plant in base_params:
    generated_parameters = technology_to_function[plant](base_params[plant])  # Use the corrected key here
    # Assume EngageAnalysis is a previously defined class for analysis
    engage_analysis = geophires_parametrization_analysis(plant,target_prod_temp_min, target_prod_temp_max)

    # Prepare parameters for analysis (Assuming this method is defined in EngageAnalysis)
    engage_analysis.prepare_parameters(generated_parameters)

    print(f"============System Configured for {environment}-{technology} Plant=================\n")
    # Assuming pprint is imported and used for pretty-printing
    pprint.pprint(generated_parameters[0])
    print(f"\n===================================================================================")
else:
    print(f"Combination '{plant}' is not supported.")


In [ ]:
# # Run iterations and process the data
engage_analysis.run_iterations()

In [ ]:
df_final = engage_analysis.get_final_dataframe()
# Remove rows where any of the values are negative
df_final = df_final[(df_final >= 0).all(axis=1)]
# Check if the dataframe is empty after filtering
if df_final.empty:
    print("============================================================================================")
    print("Consider revising the target temperature range, as achieving the specified temperatures may not be feasible with ", plant)
    print("============================================================================================")

# Sorting and saving to Excel
df_final = df_final.sort_values(
    by=['Depth (m)', 'Number of Prod Wells', 'Number of Inj Wells'], ascending=[True, True, True]
)
df_final.to_csv(f'results/{plant}_results.csv')

df_final.head()

In [ ]:
# Function to safely extract a column as an array, or fill with zeros if not present
def safe_extract(df, column_name):
    if column_name in df.columns:
        return np.array(df[column_name])
    else:
        print(f"'{column_name}' not found in DataFrame. Filling with zeros.")
        return np.zeros(len(df))

# Prepare data for plots
cmap = plt.get_cmap('plasma')
unique_prod_wells = df_final['Number of Prod Wells'].unique()
unique_inj_wells = df_final['Number of Inj Wells'].unique()
unique_depth = df_final['Depth (m)'].unique()
unique_temp = df_final['Average Production Temperature (degC)'].unique()

# Safe extraction of columns
heating_capacity = safe_extract(df_final, 'Average Heat Production (MWth)')
electric_capacity = safe_extract(df_final, 'Average Electricity Production (MWe)')
surface_cost = safe_extract(df_final, 'Surface Plant Cost ($M)')
surface_o_m_cost = safe_extract(df_final, 'Surface maintenance costs ($MUSD/yr)')
# eff = safe_extract(df_final, 'Eff ($MUSD/yr)')

# Reservoir related data
thermal_capacity = safe_extract(df_final, 'Average Reservoir Heat Extraction (MWth)')
reservoir_cost = safe_extract(df_final, 'Drilling and completion cost ($MUSD)')
reservoir_o_m_cost = safe_extract(df_final, 'Wellfield maintenance costs ($MUSD/yr)') + safe_extract(
    df_final, 'Make-Up Water O&M Cost ($MUSD/year)'
)

# Total cost
total_cap_cost = safe_extract(df_final, 'Total capital costs ($MUSD/yr)')

# Function to check if data is all zeros or empty
def has_data(*args):
    return all(not np.all(arg == 0) and len(arg) > 0 for arg in args)

# # Fit linear models for eachship
#fit_linear_model(x, y, residual, slope offset, y intercept offset)  
a1, b1, x1_line, lower_b1_line, label_b1 = fit_linear_model(heating_capacity, surface_cost, 2,-0.05,2)         # heating cap vs surface cost
a2, b2, x2_line, lower_b2_line, label_b2 = fit_linear_model(electric_capacity, surface_cost, 2,0,0)         # electric cap vs surface cost

a3, b3, x3_line, lower_b3_line, label_b3 = fit_linear_model(electric_capacity, surface_o_m_cost, 2,0,0)     # electric cap vs surface O&M cost
a4, b4, x4_line, lower_b4_line, label_b4 = fit_linear_model(heating_capacity, surface_o_m_cost, 2,-0.15,-0.05)      # heating cap vs surface O&M cost

# Reservoir
a5, b5, x5_line, lower_b5_line, label_b5 = fit_linear_model(thermal_capacity, reservoir_cost, 2,0,0.5)        # thermal cap vs reservoir cost
a6, b6, x6_line, lower_b6_line, label_b6 = fit_linear_model(thermal_capacity, reservoir_o_m_cost, 2,-0.05,-0.05)    # thermal cap vs reservoir O&M cost

##Total
# a7, b7, x7_line, lower_b7_line, label_b7 = fit_linear_model(electric_capacity, total_cap_cost, 0,0,0)         # heating cap vs total cost
# a8, b8, x8_line, lower_b8_line, label_b8 = fit_linear_model(heating_capacity, total_cap_cost, 0,0,0)         # heating cap vs total cost
# a9, b9, x9_line, lower_b9_line, label_b9 = fit_linear_model(thermal_capacity, total_cap_cost, 0,0,0)         # thermal cap vs total cost

# Create scatter plots for each relationship
# Heating Capacity vs Surface Cost
if has_data(heating_capacity, surface_cost):
    create_scatter_plot(
        heating_capacity,
        surface_cost,
        x1_line,
        lower_b1_line,
        label_b1,
        f'(A) {plant} surface cost-to-heating capacity',
        'Avg. Heating capacity (MWth)',
        'Surface Total Cost ($M)',
        cmap,
        unique_prod_wells,
        unique_inj_wells,
        unique_depth,
        unique_temp,
        df_final,
    )

# Electric Capacity vs Surface Cost
if has_data(electric_capacity, surface_cost):
    create_scatter_plot(
        electric_capacity,
        surface_cost,
        x2_line,
        lower_b2_line,
        label_b2,
        f'(B) {plant} surface cost-to-electric capacity',
        'Avg. Electric capacity (MWe)',
        'Surface Total Cost ($M)',
        cmap,
        unique_prod_wells,
        unique_inj_wells,
        unique_depth,
        unique_temp,
        df_final
    )

## Electric Capacity vs Surface O&M Cost
if has_data(electric_capacity, surface_o_m_cost):
    create_scatter_plot(
        electric_capacity,
        surface_o_m_cost,
        x3_line,
        lower_b3_line,
        label_b3,
        f'(C) {plant} surface O&M cost-to-electric capacity',
        'Avg. Electric capacity (MWe)',
        'Surface O&M Total Cost ($M)',
        cmap,
        unique_prod_wells,
        unique_inj_wells,
        unique_depth,
        unique_temp,
        df_final
    )

# # Heating Capacity vs Surface O&M Cost
if has_data(heating_capacity, surface_o_m_cost):
    create_scatter_plot(
        heating_capacity,
        surface_o_m_cost,
        x4_line,
        lower_b4_line,
        label_b4,
        f'(D) {plant} surface O&M cost-to-heating capacity',
        'Avg. Heating capacity (MWth)',
        'Surface O&M Total Cost ($M)',
        cmap,
        unique_prod_wells,
        unique_inj_wells,
        unique_depth,
        unique_temp,
        df_final,
    )

## Thermal Capacity vs Reservoir Cost
if has_data(thermal_capacity, reservoir_cost):
    create_scatter_plot(
        thermal_capacity,
        reservoir_cost,
        x5_line,
        lower_b5_line,
        label_b5,
        f'(E) {plant} reservoir cost-to-thermal capacity',
        'Avg. Thermal capacity (MWth)',
        'Reservoir Total Cost ($M)',
        cmap,
        unique_prod_wells,
        unique_inj_wells,
        unique_depth,
        unique_temp,
        df_final,
    )

## Thermal Capacity vs Reservoir O&M Cost
if has_data(thermal_capacity, reservoir_o_m_cost):
    create_scatter_plot(
        thermal_capacity,
        reservoir_o_m_cost,
        x6_line,
        lower_b6_line,
        label_b6,
        f'(F) {plant} reservoir O&M cost-to-thermal capacity',
        'Avg. Thermal capacity (MWth)',
        'Reservoir O&M Total Cost ($M)',
        cmap,
        unique_prod_wells,
        unique_inj_wells,
        unique_depth,
        unique_temp,
        df_final,
    )

save_image(f'results/{plant}_plots.pdf')

In [ ]:
slope_values = [a1, a2, a3, a4, a5, a6]

# Convert slopes from $/MW to $/kW by multiplying each element by 1000, 
# and replace 1000.000 with NaN to indicate analysis was not applicable
slope_dollars_per_kw = [value * 1000 if value != 1 else np.nan for value in slope_values]

data = {
    'Relationship Name': [
        'Heating Capacity vs Surface Cost ($/kW)',
        'Electric Capacity vs Surface Cost ($/kW)',
        'Electric Capacity vs Surface O&M Cost ($/kW)',
        'Heating Capacity vs Surface O&M Cost ($/kW)',
        'Thermal Capacity vs Reservoir Cost ($/kW)',
        'Thermal Capacity vs Reservoir O&M Cost ($/kW)'
    ],
    'Value': slope_dollars_per_kw
}

# Create the initial DataFrame
df = pd.DataFrame(data)

# Add the max average production temperature as a new row using pd.concat
max_values = {
    'Max Total Electricity Generation (MWe)': df_final['Maximum Total Electricity Generation (MWe)'].max()*2,
    # 'Max Cooling Production (MW)': df_final['Maximum Cooling Production (MW)'].max(),
    'Max Net Heat Production (MWth)': df_final['Maximum Net Heat Production (MWth)'].max()*2,
    'Max Avg Reservoir Heat Extraction (MWth)': df_final['Average Reservoir Heat Extraction (MWth)'].max()*2
}

# Append each max value as a new row in `df`
for relationship_name, max_value in max_values.items():
    new_row = pd.DataFrame({'Relationship Name': [relationship_name], 'Value': [max_value]})
    df = pd.concat([df, new_row], ignore_index=True)

df.round(2)